# SarXarray Demo Notebook

In this Jupyter Notebook, we demonstrate the following operations using `sarxarray`:

- Load an SLC stack in binary format into a `xarray.Dataset` object;
- Append variable data fields;
- Create an MRM of a subset of the SLC stack;
- Apply point selection to a subset of the SLC stack;
- Export the selection results in Zarr format

The data needed for this Notebook is sored in the following pth on Spider: `/project/caroline/Public/demo_sarxarray`.

In [1]:
# To execute this notebook, you may need to install matplotlib
# For example, execute this block to install matplotlib into "sarxarray" environment
# ! conda install -n sarxarray -c conda-forge matplotlib -y

In [1]:
import numpy as np
from pathlib import Path
import sarxarray

### Load a SLC stack

In [2]:
# You many need to change this cell to your local data directory
# path = Path('../../data/nl_veenweiden_s1_asc_t088/')
path = Path('/project/caroline//Share/stacks/nl_veenweiden_s1_asc_t088/stack')

In [27]:
# Make a list of SLCs to read
f_slc = 'cint.raw' 
list_slcs = [p/f_slc for p in path.rglob(('[0-9]' * 8)) if not p.match('20200325')] # cint for master is missing
# list_slcs

In [28]:
len(list_slcs)

374

In [29]:
# Read metadata
shape=(10018, 68656)
dtype = np.dtype([('re', np.float32), ('im', np.float32)])

In [30]:
stack = sarxarray.from_binary(list_slcs, shape, dtype=dtype)
stack

<xarray.Dataset>
Dimensions:    (azimuth: 10018, range: 68656, time: 374)
Coordinates:
  * azimuth    (azimuth) int64 0 1 2 3 4 5 ... 10013 10014 10015 10016 10017
  * range      (range) int64 0 1 2 3 4 5 ... 68650 68651 68652 68653 68654 68655
  * time       (time) int64 0 1 2 3 4 5 6 7 ... 366 367 368 369 370 371 372 373
Data variables:
    complex    (azimuth, range, time) complex64 dask.array<chunksize=(50, 68656, 1), meta=np.ndarray>
    amplitude  (azimuth, range, time) float16 dask.array<chunksize=(50, 68656, 1), meta=np.ndarray>
    phase      (azimuth, range, time) float16 dask.array<chunksize=(50, 68656, 1), meta=np.ndarray>

In [10]:
stack.complex

<xarray.DataArray 'complex' (azimuth: 10018, range: 68656, time: 375)>
dask.array<transpose, shape=(10018, 68656, 375), dtype=complex64, chunksize=(50, 68656, 1), chunktype=numpy.ndarray>
Coordinates:
  * azimuth  (azimuth) int64 0 1 2 3 4 5 ... 10012 10013 10014 10015 10016 10017
  * range    (range) int64 0 1 2 3 4 5 6 ... 68650 68651 68652 68653 68654 68655
  * time     (time) int64 0 1 2 3 4 5 6 7 8 ... 367 368 369 370 371 372 373 374

### Create MRM of a subset

Here you can use the dask plugin on the left side of your Jupyter panel, and spin up workers

In [31]:
from dask.distributed import Client

client = Client("tcp://10.0.2.20:44789")
client

<Client: 'tcp://10.0.2.20:44789' processes=4 threads=16, memory=120.00 GiB>

In [32]:
mrm = stack.slcstack.mrm()
mrm

<xarray.DataArray 'amplitude' (azimuth: 10018, range: 68656)>
dask.array<mean_agg-aggregate, shape=(10018, 68656), dtype=float16, chunksize=(50, 68656), chunktype=numpy.ndarray>
Coordinates:
  * azimuth  (azimuth) int64 0 1 2 3 4 5 ... 10012 10013 10014 10015 10016 10017
  * range    (range) int64 0 1 2 3 4 5 6 ... 68650 68651 68652 68653 68654 68655

In [33]:
# Compute a subset of the mrm
mrm_subset = mrm[5000:6000, 10000:11000]
mrm_subset = mrm_subset.compute()

In [34]:
# Visualize
from matplotlib import pyplot as plt
fig, ax = plt.subplots()
ax.imshow(mrm_subset)
mrm_subset.plot(robust=True, ax=ax)

ModuleNotFoundError: No module named 'matplotlib'